In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import lsst.daf.butler as dafButler

plt.set_loglevel('WARNING')

INFO:numexpr.utils:Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
# Load collection
repo = '/repo/main'
instrument = "LSSTComCam"
butler= dafButler.Butler(repo, collections='u/bos/DM-48672/LSSTComCam_drp_association_v2', instrument=instrument)
butler_w04= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/w_2025_04/DM-48556', instrument=instrument)

In [3]:
lc_refs = butler.query_datasets("diaSourceTable_tract")
obj_refs = butler.query_datasets("diaObjectTable_tract")

print(f"Number of tracts in diaSourceTable (current): {len(lc_refs)}")
print(f"Number of tracts in diaObjectTable_tract (current): {len(obj_refs)}")


Number of tracts in diaSourceTable (current): 28
Number of tracts in diaObjectTable_tract (current): 28


In [4]:
# List of tract indices to process
tract_indices = np.arange(0, len(obj_refs))

# Initialize lists to hold diaObj and diaSource DataFrames
dia_Obj_list = []
diaSource_list = []

# Loop through the specified tracts and fetch the data
for idx in tqdm(tract_indices):
    obj_ref = obj_refs[idx]  # Select the corresponding obj_ref
    diaObject_tract = butler.get(obj_ref)  # Fetch diaObj for this tract
    diaSource_tract = butler.get('diaSourceTable_tract', dataId=obj_ref.dataId)  # Fetch diaSource for this tract
    
    # Append the results to the lists
    dia_Obj_list.append(diaObject_tract)
    diaSource_list.append(diaSource_tract)

# Concatenate all diaObj and diaSource DataFrames
combined_dia_Obj = pd.concat(dia_Obj_list, ignore_index=False)
combined_diaSource = pd.concat(diaSource_list, ignore_index=False)

diaObject = combined_dia_Obj.copy()
diaSource = combined_diaSource.copy()


  0%|                                                                                                                         | 0/28 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:09<00:00,  3.08it/s]


In [5]:
# List of tract indices to process
tract_indices = np.arange(0, len(obj_refs))

# Initialize lists to hold diaObj and diaSource DataFrames
dia_Obj_list = []
diaSource_list = []

# Loop through the specified tracts and fetch the data
for idx in tqdm(tract_indices):
    obj_ref = obj_refs[idx]  # Select the corresponding obj_ref
    diaObject_tract = butler_w04.get(obj_ref)  # Fetch diaObj for this tract
    diaSource_tract = butler_w04.get('diaSourceTable_tract', dataId=obj_ref.dataId)  # Fetch diaSource for this tract
    
    # Append the results to the lists
    dia_Obj_list.append(diaObject_tract)
    diaSource_list.append(diaSource_tract)

# Concatenate all diaObj and diaSource DataFrames
combined_dia_Obj = pd.concat(dia_Obj_list, ignore_index=False)
combined_diaSource = pd.concat(diaSource_list, ignore_index=False)

diaObject_04 = combined_dia_Obj.copy()
diaSource_04 = combined_diaSource.copy()

 11%|████████████                                                                                                     | 3/28 [00:00<00:04,  5.62it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:09<00:00,  3.01it/s]


In [6]:
diaSource_04

,visit,detector,band,diaObjectId,ssObjectId,parentDiaSourceId,midpointMjdTai,bboxSize,time_processed,ra,...,shape_flag_parent_source,trail_flag_edge,pixelFlags_streak,pixelFlags_streakCenter,pixelFlags_injected,pixelFlags_injectedCenter,pixelFlags_injected_template,pixelFlags_injected_templateCenter,coord_ra,coord_dec
diaSourceId,,,,,,,,,,,,,,,,,,,,,
191543819395661948,2024112700091,8,g,4496109754162085889,0,0,60642.055203,26,2025-01-27 03:13:42.385516,38.196705,...,False,False,False,False,False,False,False,False,38.196705,6.039462
191508664014602432,2024112600118,8,g,4496114152208596993,0,0,60641.062221,38,2025-01-27 03:02:18.126798,38.169583,...,False,False,False,False,False,False,False,False,38.169583,6.038664
191508664014602454,2024112600118,8,g,4496114152208596994,0,0,60641.062221,20,2025-01-27 03:02:18.126798,38.139662,...,False,False,True,True,False,False,False,False,38.139662,6.039212
191508664014602461,2024112600118,8,g,4496114152208596995,0,0,60641.062221,20,2025-01-27 03:02:18.126798,38.137379,...,False,False,True,True,False,False,False,False,38.137379,6.035444
191508664014602463,2024112600118,8,g,4496114152208596996,0,0,60641.062221,16,2025-01-27 03:02:18.126798,38.136624,...,False,False,True,True,False,False,False,False,38.136624,6.034337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192001658580893798,2024121000503,0,r,2333563896373182493,0,0,60655.293063,24,2025-01-27 03:04:36.050147,94.428439,...,False,False,False,False,False,False,False,False,94.428410,-25.221041
192001658580893799,2024121000503,0,r,2333563896373182500,0,0,60655.293063,28,2025-01-27 03:04:36.050147,94.428596,...,False,False,False,False,False,False,False,False,94.428573,-25.223499
192001658580893803,2024121000503,0,r,2333563896373182816,0,0,60655.293063,19,2025-01-27 03:04:36.050147,94.431324,...,False,False,False,False,False,False,False,False,94.431324,-25.219196


In [7]:
# Count the number of unique diaObjectId values for each diaSourceId
object_count_per_source = diaSource.groupby("diaSourceId")["diaObjectId"].nunique()

# Convert to DataFrame for easier handling
object_count_df = object_count_per_source.reset_index()
object_count_df.columns = ["diaSourceId", "num_unique_diaObjectId"]


# Count the number of unique diaObjectId values for each diaSourceId
object_count_per_source_04 = diaSource_04.groupby("diaSourceId")["diaObjectId"].nunique()

# Convert to DataFrame for easier handling
object_count_df_04 = object_count_per_source_04.reset_index()
object_count_df_04.columns = ["diaSourceId", "num_unique_diaObjectId"]


In [8]:
# diaSources that are associated with more than one diaObject in the new version
object_count_df[object_count_df['num_unique_diaObjectId']>1]

,diaSourceId,num_unique_diaObjectId
1,190875481648660482,2
4,190875481648660485,3
5,190875481648660486,3
7,190875481648660488,2
10,190875481648660491,3
...,...,...
5083565,192036958950655465,2
5083566,192036958950655466,2
5083567,192036958950655467,2
5083568,192036958950655468,2


In [9]:
diaSource.loc[191403211800707126]

,visit,detector,band,diaObjectId,ssObjectId,parentDiaSourceId,midpointMjdTai,bboxSize,time_processed,ra,...,shape_flag_parent_source,trail_flag_edge,pixelFlags_streak,pixelFlags_streakCenter,pixelFlags_injected,pixelFlags_injectedCenter,pixelFlags_injected_template,pixelFlags_injected_templateCenter,coord_ra,coord_dec
diaSourceId,,,,,,,,,,,,,,,,,,,,,
191403211800707126,2024112300212,1,r,4602546877777313793,0,0,60638.120204,19,2025-01-27 03:10:12.958658,38.177778,...,False,False,False,False,False,False,False,False,38.177778,7.518649
191403211800707126,2024112300212,1,r,4708192353020543014,0,0,60638.120204,19,2025-01-27 03:10:12.958658,38.177778,...,False,False,False,False,False,False,False,False,38.177778,7.518649
191403211800707126,2024112300212,1,r,4602071888754114594,0,0,60638.120204,19,2025-01-27 03:10:12.958658,38.177778,...,False,False,False,False,False,False,False,False,38.177778,7.518649
191403211800707126,2024112300212,1,r,4707712965950832680,0,0,60638.120204,19,2025-01-27 03:10:12.958658,38.177778,...,False,False,False,False,False,False,False,False,38.177778,7.518649


In [10]:

# diaSources that are associated with more than one diaObject in the weekly 03
object_count_df_04[object_count_df_04['num_unique_diaObjectId']>1]

,diaSourceId,num_unique_diaObjectId
1,190875481648660482,2
4,190875481648660485,3
5,190875481648660486,3
7,190875481648660488,2
10,190875481648660491,3
...,...,...
5083565,192036958950655465,2
5083566,192036958950655466,2
5083567,192036958950655467,2
5083568,192036958950655468,2


In [11]:
diaSource_04.loc[191403211800707126]

,visit,detector,band,diaObjectId,ssObjectId,parentDiaSourceId,midpointMjdTai,bboxSize,time_processed,ra,...,shape_flag_parent_source,trail_flag_edge,pixelFlags_streak,pixelFlags_streakCenter,pixelFlags_injected,pixelFlags_injectedCenter,pixelFlags_injected_template,pixelFlags_injected_templateCenter,coord_ra,coord_dec
diaSourceId,,,,,,,,,,,,,,,,,,,,,
191403211800707126,2024112300212,1,r,4602546877777313793,0,0,60638.120204,19,2025-01-27 03:10:12.958658,38.177778,...,False,False,False,False,False,False,False,False,38.177778,7.518649
191403211800707126,2024112300212,1,r,4708192353020543014,0,0,60638.120204,19,2025-01-27 03:10:12.958658,38.177778,...,False,False,False,False,False,False,False,False,38.177778,7.518649
191403211800707126,2024112300212,1,r,4602071888754114594,0,0,60638.120204,19,2025-01-27 03:10:12.958658,38.177778,...,False,False,False,False,False,False,False,False,38.177778,7.518649
191403211800707126,2024112300212,1,r,4707712965950832680,0,0,60638.120204,19,2025-01-27 03:10:12.958658,38.177778,...,False,False,False,False,False,False,False,False,38.177778,7.518649
